### 다음 데이터 크롤링 (2021-03-07)
#### 웹툰정보 크롤링
* 평점순 웹툰 : http://m.webtoon.daum.net/data/mobile/webtoon?sort=score&page_size=9999&page_no=1
* 웹툰 이름, 장르, 작가, 게시일, url, 줄거리, 평점, 19금여부, 완결여부 크롤링

In [1]:
from IPython.display import clear_output # clear_output() 으로 아웃풋 제거 가능
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)

In [2]:
# scrapy
!pip install scrapy

     |████████████████████████████████| 245kB 5.5MB/s 
     |████████████████████████████████| 3.2MB 7.3MB/s 
     |████████████████████████████████| 61kB 5.5MB/s 
     |████████████████████████████████| 3.2MB 30.1MB/s 
     |████████████████████████████████| 245kB 40.7MB/s 
     |████████████████████████████████| 3.1MB 47.5MB/s 
     |████████████████████████████████| 81kB 8.0MB/s 
  Created wheel for protego: filename=Protego-0.1.16-cp37-none-any.whl size=7766 sha256=45995b2922ff2d24c306ed5a83b84d7c3ceccd8443c61f442fa04d6e7f90e8fd
  Stored in directory: /root/.cache/pip/wheels/51/01/d1/4a2286a976dccd025ba679acacfe37320540df0f2283ecab12
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp37-none-any.whl size=11517 sha256=07ebc138e812f3c176f0eb1e147b1765aa188aaae47f3791fe898b74168cc199
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
Successfully built protego PyDispatcher


In [3]:
import scrapy
import requests
from scrapy.http import TextResponse
import time
import urllib
import urllib3

In [4]:
from pandas.io.json import json_normalize

# 데이터를 가져와서 판다스의 데이터프레임으로 만드는 함수
def get_data(url):
    response = requests.get(url)
    json_info = response.json() # json -> dict 형식으로 알아서 바꿔줌
    
    return json_info

In [5]:
# 19금 만화도 크롤링하려면 로그인을 해야한다

import getpass

myid = getpass.getpass('input your id : ')
mypassword = getpass.getpass('input your password : ')

LOGIN_URL = 'https://logins.daum.net/accounts/loginform.do'
LOGIN_DATA = {
    'id': myid,
    'inputPwd': mypassword
}
with requests.Session() as s:
    res = s.post(LOGIN_URL, data=LOGIN_DATA, verify=False, allow_redirects=False)

    print(res.status_code)

input your id : ··········
input your password : ··········
200


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [6]:
# 평점순 웹툰url
# url = 'http://m.webtoon.daum.net/data/mobile/webtoon?sort=score&page_size=50&page_no=1&1615108397290'

# -> page_size=9999(9999번까지) & page_no=1(1페이지로)
url1 = 'http://m.webtoon.daum.net/data/mobile/webtoon?sort=score&page_size=9999&page_no=1'
wbtn_raw1 = get_data(url1)
# wbtn_raw1

In [7]:
wbt1_df = pd.DataFrame(columns = ['id', 'nickname', 'webtoonType', 'title', 'englishTitle', 'finishYn', 'titleImage2', 'thumbnailImage', 'thumbnailImage2', 'padThumbnailImage', 'artistCommentImage', 'homeThumbnailImage', 'appThumbnailImage', 'pcThumbnailImage', 'pcRecommendImage', 'pcHomeImage', 'pcHomeRightImage', 'pcHomeLeftColor', 'pcHomeRightColor', 'introduction', 'startDate', 'finishDate', 'restartDate', 'serviceStatus', 'viewerType', 'weekTerm', 'articleId', 'media', 'url', 'simpleUrl', 'webtoonGroupId', 'payYn', 'payType', 'price', 'ageGrade', 'restYn', 'monopolize', 'dateCreated', 'webtoonComment', 'cartoon', 'cp', 'webtoonWeeks', 'webtoonEpisodes', 'previewWebtoonEpisodes', 'latestWebtoonEpisode', 'webtoonServices', 'relatedProducts', 'promotionContents', 'score', 'tag', 'isNew', 'averageScore', 'seriesYn', 'ranking', 'diff', 'metricsScore', 'eventBanners', 'sort', 'sortWeight', 'gidamoo', 'webtoonNotice', 'couponAvailable'])

for data in wbtn_raw1['data']['webtoons']:
    wbt1_df.loc[len(wbt1_df)] = data

# wbt1_df.head(20)

In [ ]:
# # NaN,None값으로만 이루어진 column
# nulldf = wbt1_df.isnull().sum()>=603
# nulldf[nulldf==False].keys()

한 개의 column선택
* df["account"]

1개 이상의 column 선택
* df[["account", "street", "state"]]

In [ ]:
# chkdf = wbt1_df['isNew']==False
# len(chkdf[chkdf==True].keys())

In [8]:
wbt1_df2 = wbt1_df[['id', 'nickname', #웹툰 닉네임
                    'title', 'finishYn', 'thumbnailImage2', #'appThumbnailImage', 
                    #'viewerType', #(1206개중 1172개가 scroll) 'articleId', 'price' (1206개 다 0), 
                    'ageGrade', # (1206개중 1004개 0) #'restYn', # (1206개중 1153개가 'N') -> 휴재 여부인 듯
                    'dateCreated', 'cartoon', 'latestWebtoonEpisode', 'webtoonServices', 'averageScore'
                    # 'score',(1206개 다 0) #'isNew', # 신작 여부인 듯 'ranking', 'diff', 'metricsScore', 'sortWeight', 'couponAvailable'
        ]]
# wbt1_df2

In [9]:
# 썸네일 이미지 링크 정제

new_temp = pd.DataFrame(columns=['url'])
for data in wbt1_df2['thumbnailImage2']:
    new_temp.loc[len(new_temp)] = data['url']
# new_temp.tail()

In [10]:
# 글/그림 작가 모으기

new_temp2 = pd.DataFrame(columns=['artist_name'])
for data in wbt1_df2['cartoon']:
    new_temp2.loc[len(new_temp2)] = data['artists'][0]['name']+'/'+data['artists'][1]['name']
# new_temp2

In [11]:
# 마지막 업로드 일자/ 현재(완료) 회차

new_temp3 = pd.DataFrame(columns=['count_chapter'])
new_temp4 = pd.DataFrame(columns=['latest_Created'])
for data in wbt1_df2['latestWebtoonEpisode']:
    try:
        new_temp3.loc[len(new_temp3)]= data['episode']
    except:
        new_temp3.loc[len(new_temp3)]= None
    try:
        new_temp4.loc[len(new_temp4)]= data['dateCreated']
    except:
        new_temp4.loc[len(new_temp4)]= None     
# new_temp3
# new_temp4

In [12]:
df = wbt1_df2[['id', 'nickname', 'title', 'finishYn', 'ageGrade', 'dateCreated', 'averageScore']]
df['artist']=new_temp2
df['latest_upload']=new_temp4
df['last_chapter']=new_temp3
df['img_url']=new_temp
# df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [13]:
# 장르/카테고리, 소개글, 자동수집

intro = []
genres = []
categories = []
print('trying to getting datas from Daum Webtoon...', end='')
for i, data in df.iterrows():
    url2 = 'http://webtoon.daum.net/data/pc/webtoon/view/'+data['nickname']
    temp_df= get_data(url2)
    try:
        intro.append(temp_df['data']['webtoon']['introduction'])
    except:
        intro.append(None)
    gs=[]
    try:
        for data in temp_df['data']['webtoon']['cartoon']['genres']:
            gs.append(data['name'])
        gs_text = '/'.join(gs)
        genres.append(gs_text)
    except:
        genres.append(None)
    ls = []
    try:
        for data in temp_df['data']['webtoon']['cartoon']['categories']:
            ls.append(data['name'])
        tmp_text = '/'.join(ls)
        categories.append(tmp_text)
    except:
        categories.append(None)
    print(i,".", end='')
print("Crawling complete!")

df['intro'] = pd.Series(intro)
df['genres'] = pd.Series(genres)
df['categories'] = pd.Series(categories)
# df

trying to getting datas from Daum Webtoon...0 .1 .2 .3 .4 .5 .6 .7 .8 .9 .10 .11 .12 .13 .14 .15 .16 .17 .18 .19 .20 .21 .22 .23 .24 .25 .26 .27 .28 .29 .30 .31 .32 .33 .34 .35 .36 .37 .38 .39 .40 .41 .42 .43 .44 .45 .46 .47 .48 .49 .50 .51 .52 .53 .54 .55 .56 .57 .58 .59 .60 .61 .62 .63 .64 .65 .66 .67 .68 .69 .70 .71 .72 .73 .74 .75 .76 .77 .78 .79 .80 .81 .82 .83 .84 .85 .86 .87 .88 .89 .90 .91 .92 .93 .94 .95 .96 .97 .98 .99 .100 .101 .102 .103 .104 .105 .106 .107 .108 .109 .110 .111 .112 .113 .114 .115 .116 .117 .118 .119 .120 .121 .122 .123 .124 .125 .126 .127 .128 .129 .130 .131 .132 .133 .134 .135 .136 .137 .138 .139 .140 .141 .142 .143 .144 .145 .146 .147 .148 .149 .150 .151 .152 .153 .154 .155 .156 .157 .158 .159 .160 .161 .162 .163 .164 .165 .166 .167 .168 .169 .170 .171 .172 .173 .174 .175 .176 .177 .178 .179 .180 .181 .182 .183 .184 .185 .186 .187 .188 .189 .190 .191 .192 .193 .194 .195 .196 .197 .198 .199 .200 .201 .202 .203 .204 .205 .206 .207 .208 .209 .210 .211 .212 .2

In [14]:
# 웹툰 url

w_url = []
for i, data in df.iterrows():
    w_url.append('http://webtoon.daum.net/webtoon/view/'+data['nickname'])

df['page url'] = pd.Series(w_url)
# df

In [15]:
df.to_csv("D_Webtoon_1_toonsinfo.csv",index=False)